# Deep Learning Applications: Laboratory #1

In this first laboratory we will work relatively simple architectures to get a feel for working with Deep Models. This notebook is designed to work with PyTorch, but as I said in the introductory lecture: please feel free to use and experiment with whatever tools you like.

**Important Notes**:
1. Be sure to **document** all of your decisions, as well as your intermediate and final results. Make sure your conclusions and analyses are clearly presented. Don't make us dig into your code or walls of printed results to try to draw conclusions from your code.
2. If you use code from someone else (e.g. Github, Stack Overflow, ChatGPT, etc) you **must be transparent about it**. Document your sources and explain how you adapted any partial solutions to creat **your** solution.



-----
## Exercise 2: Choose at Least One

Below are **three** exercises that ask you to deepen your understanding of Deep Networks for visual recognition. You must choose **at least one** of the below for your final submission -- feel free to do **more**, but at least **ONE** you must submit. Each exercise is designed to require you to dig your hands **deep** into the guts of your models in order to do new and interesting things.

**Note**: These exercises are designed to use your small, custom CNNs and small datasets. This is to keep training times reasonable. If you have a decent GPU, feel free to use pretrained ResNets and larger datasets (e.g. the [Imagenette](https://pytorch.org/vision/0.20/generated/torchvision.datasets.Imagenette.html#torchvision.datasets.Imagenette) dataset at 160px).

### Exercise 2.1: *Fine-tune* a pre-trained model
Train one of your residual CNN models from Exercise 1.3 on CIFAR-10. Then:
1. Use the pre-trained model as a **feature extractor** (i.e. to extract the feature activations of the layer input into the classifier) on CIFAR-100. Use a **classical** approach (e.g. Linear SVM, K-Nearest Neighbor, or Bayesian Generative Classifier) from scikit-learn to establish a **stable baseline** performance on CIFAR-100 using the features extracted using your CNN.
2. Fine-tune your CNN on the CIFAR-100 training set and compare with your stable baseline. Experiment with different strategies:
    - Unfreeze some of the earlier layers for fine-tuning.
    - Test different optimizers (Adam, SGD, etc.).

Each of these steps will require you to modify your model definition in some way. For 1, you will need to return the activations of the last fully-connected layer (or the global average pooling layer). For 2, you will need to replace the original, 10-class classifier with a new, randomly-initialized 100-class classifier.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from types import SimpleNamespace
from tqdm import tqdm
import copy

from src.trainer import trainer, trainer_ft
from src.dataloader import get_cifar10_loaders, get_cifar100_loaders
from src.config import get_config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

parser = get_config()
config = parser.parse_args([])

In [2]:
# RESNET ADAPTATION FOR CIFAR-10
def get_resnet18_for_cifar(num_classes=10):
    model = models.resnet18(weights=None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

#### 1 - Pre-training on CIFAR-10

In [3]:
cifar10_train_loader, cifar10_val_loader, _ = get_cifar10_loaders(config)
pretrained_model = get_resnet18_for_cifar(num_classes=10).to(device)
pretrain_save_path = "models2/resnet18_cifar10.pth"

# Aggiorna la config per questo specifico training
pretrain_config = parser.parse_args([])
pretrain_config.max_epoch = 100
pretrain_config.lr = 0.01
pretrain_config.batch_size = 128
pretrain_config.optimizer = 'Adam'
pretrain_config.dataset = 'CIFAR10'

trainer_ft(
    model=pretrained_model,
    dl_train=cifar10_train_loader,
    dl_val=cifar10_val_loader,
    device=device,
    save_path=pretrain_save_path,
    config_args=pretrain_config
)

CIFAR-10 DataLoaders pronti.
  - Training samples: 45000, Validation samples: 5000, Test samples: 10000


wandb: Currently logged in as: alessandraspin (alessandraspin-universit-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


352 iterations per epoch

--- STARTING TRAINING of ResNet ---


Training ResNet:   1%|▎                         | 1/100 [00:17<29:31, 17.90s/it]

Epoch 1: New best model saved to models2/resnet18_cifar10.pth with Val Acc: 0.3904


...

Epoch 51: New best model saved to models2/resnet18_cifar10.pth with Val Acc: 0.8892


Training ResNet: 100%|████████████████████████| 100/100 [27:48<00:00, 16.69s/it]


train/loss,█▅▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,█▇▇▆▆▄▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▄▄▅▆▆▇▇████████████████████████████████
val/loss,█▆▄▆▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,0.1395
train/lr,0.0
val/accuracy,0.8872
val/loss,0.38111



--- FINISHED TRAINING. Best Val Acc: 0.8892 ---


0.8892

#### 2 - Feature Extractor and baseline

In [4]:
cifar100_train_loader, cifar100_val_loader, cifar100_test_loader = get_cifar100_loaders(config)

feature_extractor = get_resnet18_for_cifar(num_classes=10)
feature_extractor.load_state_dict(torch.load(pretrain_save_path))
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1]).to(device).eval()

def extract_features(loader, model, device):
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc="Extracting features"):
            feats = model(inputs.to(device)).view(inputs.size(0), -1)
            features.append(feats.cpu().numpy())
            labels.append(targets.cpu().numpy())
    return np.concatenate(features), np.concatenate(labels)

X_train, y_train = extract_features(cifar100_train_loader, feature_extractor, device)
X_test, y_test = extract_features(cifar100_test_loader, feature_extractor, device)

Caricamento dei dati CIFAR-100...
CIFAR-100 DataLoaders pronti.
  - Training samples: 45000, Validation samples: 5000, Test samples: 10000


Extracting features: 100%|██████████| 79/79 [00:01<00:00, 56.79it/s]


In [5]:
# Baseline with Linear SVM
svm_classifier = make_pipeline(StandardScaler(), LinearSVC(dual="auto", tol=1e-5))
svm_classifier.fit(X_train, y_train)
svm_accuracy = svm_classifier.score(X_test, y_test) * 100
print(f"Baseline (Linear SVM) Accuracy: {svm_accuracy:.2f}%")

# Baseline with Linear kNN
knn_classifier = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=10))
knn_classifier.fit(X_train, y_train)
knn_accuracy = knn_classifier.score(X_test, y_test) * 100
print(f"Baseline (KNN) Accuracy: {knn_accuracy:.2f}%")

Baseline (Linear SVM) Accuracy: 34.51%
Baseline (KNN) Accuracy: 25.14%


#### 3 - Fine-Tuning on CIFAR-100

In [7]:
from torch import nn

# --- Strategy 2.1: Fine-tune only the classifier (with Adam) ---
print("\n--- Strategy 2.1: Fine-tune only the classifier (with Adam) ---")
model_ft_1 = get_resnet18_for_cifar(num_classes=10)
model_ft_1.load_state_dict(torch.load(pretrain_save_path))

# Congelo tutto
for param in model_ft_1.parameters():
    param.requires_grad = False
# Rimpiazzo la fc (di default allenabile)
model_ft_1.fc = nn.Linear(model_ft_1.fc.in_features, 100)
model_ft_1 = model_ft_1.to(device)

ft1_config = parser.parse_args([])
ft1_config.max_epoch = 30
ft1_config.optimizer = 'Adam'
ft1_config.dataset = 'CIFAR100_FT_Classifier'
ft1_config.lr = 1e-3
ft1_config.weight_decay = 5e-4

ft1_accuracy = trainer_ft(
    model_ft_1,
    cifar100_train_loader,
    cifar100_val_loader,
    device,
    "models2/ft_classifier.pth",
    ft1_config
) * 100


# --- Strategy 2.2: Unfreeze classifier + last block (with SGD) ---
print("\n--- Strategy 2.2: Fine-tuning the classifier and last block (SGD) ---")
model_ft_2 = get_resnet18_for_cifar(num_classes=10)
model_ft_2.load_state_dict(torch.load(pretrain_save_path))

# Congelo tutto
for param in model_ft_2.parameters():
    param.requires_grad = False
# Sblocco fc e layer4
model_ft_2.fc = nn.Linear(model_ft_2.fc.in_features, 100)
for param in model_ft_2.fc.parameters():
    param.requires_grad = True
for param in model_ft_2.layer4.parameters():
    param.requires_grad = True
model_ft_2 = model_ft_2.to(device)

ft2_config = parser.parse_args([])
ft2_config.max_epoch = 30
ft2_config.optimizer = 'SGD'
ft2_config.dataset = 'CIFAR100_FT_Layer4'
ft2_config.lr = 1e-2     # più alto per tutta la parte sbloccata
ft2_config.weight_decay = 5e-4

ft2_accuracy = trainer_ft(
    model_ft_2,
    cifar100_train_loader,
    cifar100_val_loader,
    device,
    "models2/ft_layer4.pth",
    ft2_config
) * 100



--- Strategy 2.1: Fine-tune only the classifier (with Adam) ---


352 iterations per epoch

--- STARTING TRAINING of ResNet ---


Training ResNet:   3%|▉                          | 1/30 [00:06<03:08,  6.49s/it]

Epoch 1: New best model saved to models2/ft_classifier.pth with Val Acc: 0.2020


Training ResNet:   7%|█▊                         | 2/30 [00:13<03:02,  6.53s/it]

...

Training ResNet:  97%|█████████████████████████▏| 29/30 [03:46<00:07,  7.78s/it]

Epoch 29: New best model saved to models2/ft_layer4.pth with Val Acc: 0.3904


Training ResNet: 100%|██████████████████████████| 30/30 [03:54<00:00,  7.80s/it]


train/loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,█▇▇▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val/accuracy,▁▂▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇█▇▇█████████
val/loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,2.29573
train/lr,0.00047
val/accuracy,0.3866
val/loss,2.2962



--- FINISHED TRAINING. Best Val Acc: 0.3904 ---


In [8]:
print("\n--- Accuracies Recap ---")
print(f"Baseline (Linear SVM) : {svm_accuracy:.2f}%")
print(f"Baseline (KNN): {knn_accuracy:.2f}%")
print(f"Fine-Tuning (only Classifier, Adam): {ft1_accuracy:.2f}%")
print(f"Fine-Tuning (Classifier + Layer4, SGD): {ft2_accuracy:.2f}%")


--- Accuracies Recap ---
Baseline (Linear SVM) : 34.51%
Baseline (KNN): 25.14%
Fine-Tuning (only Classifier, Adam): 28.04%
Fine-Tuning (Classifier + Layer4, SGD): 39.04%
